# Speculative Decoding

SGLang now provides an EAGLE-based (EAGLE-2/EAGLE-3) speculative decoding option. Our implementation aims to maximize speed and efficiency and is considered to be among the fastest in open-source LLM engines.

### Performance Highlights

Please see below for the huge improvements on throughput for LLaMA-Instruct 3.1 8B tested on MT bench that can be achieved via EAGLE3 decoding.
For further details please see the [EAGLE3 paper](https://arxiv.org/pdf/2503.01840).

| Method | Throughput (tokens/s) |
|--------|----------------|
| SGLang (w/o speculative, 1x H100) | 158.34 tokens/s |
| SGLang + EAGLE-2 (1x H100) | 244.10 tokens/s |
| SGLang + EAGLE-3 (1x H100) | 373.25 tokens/s |

## EAGLE Decoding

To enable EAGLE speculative decoding the following parameters are relevant:
* `speculative_draft_model_path`: Specifies draft model. This parameter is required.
* `speculative_num_steps`: Depth of autoregressive drafting. Increases speculation range but risks rejection cascades. Default is 5.
* `speculative_eagle_topk`: Branching factor per step. Improves candidate diversity, will lead to higher acceptance rate, but more lead to higher memory/compute consumption. Default is 4.
* `speculative_num_draft_tokens`: Maximum parallel verification capacity. Allows deeper tree evaluation but will lead to higher GPU memory usage. Default is 8.

These parameters are the same for EAGLE-2 and EAGLE-3.

You can find the best combinations of these parameters with [bench_speculative.py](https://github.com/sgl-project/sglang/blob/main/scripts/playground/bench_speculative.py).

In the documentation below, we set `--cuda-graph-max-bs` to be a small value for faster engine startup. For your own workloads, please tune the above parameters together with `--cuda-graph-max-bs`, `--max-running-requests`, `--mem-fraction-static` for the best performance. 

### EAGLE-2 decoding

You can enable EAGLE-2 decoding by setting `--speculative-algorithm EAGLE` and choosing an appropriate model.

In [1]:
from sglang.test.doc_patch import launch_server_cmd
from sglang.utils import wait_for_server, print_highlight, terminate_process

import openai

In [2]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 3 \
    --speculative-eagle-topk 4 --speculative-num-draft-tokens 16 --cuda-graph-max-bs 8
"""
)

wait_for_server(f"http://localhost:{port}")

W0827 21:04:26.800000 1340458 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0827 21:04:26.800000 1340458 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-27 21:04:27] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=35948, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.849, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=972171023, constrained_

[2025-08-27 21:04:28] Using default HuggingFace chat template with detected content format: string


W0827 21:04:35.898000 1341295 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0827 21:04:35.898000 1341295 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0827 21:04:35.908000 1341296 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0827 21:04:35.908000 1341296 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-27 21:04:36] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-08-27 21:04:36] Init torch distributed begin.


...

Capturing batches (bs=1 avail_mem=32.50 GB): 100%|██████████| 4/4 [00:09<00:00,  2.32s/it]
[2025-08-27 21:04:55] Capture draft cuda graph end. Time elapsed: 9.68 s. mem usage=5.67 GB. avail mem=32.49 GB.
[2025-08-27 21:04:55] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=32.49 GB


Capturing batches (bs=1 avail_mem=32.38 GB): 100%|██████████| 4/4 [00:00<00:00, 59.39it/s]
[2025-08-27 21:04:56] Capture draft extend cuda graph end. Time elapsed: 1.11 s. mem usage=0.10 GB. avail mem=32.38 GB.
[2025-08-27 21:04:56] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=4096, available_gpu_mem=32.38 GB


[2025-08-27 21:04:56] INFO:     Started server process [1340458]
[2025-08-27 21:04:56] INFO:     Waiting for application startup.
[2025-08-27 21:04:56] INFO:     Application startup complete.
[2025-08-27 21:04:56] INFO:     Uvicorn running on http://127.0.0.1:35948 (Press CTRL+C to quit)


[2025-08-27 21:04:57] INFO:     127.0.0.1:55702 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-27 21:04:57] INFO:     127.0.0.1:55718 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-27 21:04:57] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-27 21:04:57] INFO:     127.0.0.1:55732 - "POST /generate HTTP/1.1" 200 OK
[2025-08-27 21:04:57] The server is fired up and ready to roll!


In [3]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-27 21:05:02] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-27 21:05:02] INFO:     127.0.0.1:55748 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [4]:
terminate_process(server_process)

### EAGLE-2 Decoding with `torch.compile`

You can also enable `torch.compile` for further optimizations and optionally set `--torch-compile-max-bs`:


In [5]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --mem-fraction 0.6 \
            --enable-torch-compile --torch-compile-max-bs 2
"""
)

wait_for_server(f"http://localhost:{port}")

W0827 21:05:09.938000 1344367 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0827 21:05:09.938000 1344367 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-27 21:05:11] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=39608, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.6, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=956488137, constrained_js

[2025-08-27 21:05:11] Using default HuggingFace chat template with detected content format: string


W0827 21:05:20.593000 1344986 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0827 21:05:20.593000 1344986 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0827 21:05:20.660000 1344987 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0827 21:05:20.660000 1344987 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-27 21:05:21] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-08-27 21:05:21] Init torch distributed begin.


...

Capturing batches (bs=1 avail_mem=21.42 GB): 100%|██████████| 4/4 [00:14<00:00,  3.65s/it]
[2025-08-27 21:06:24] Capture draft cuda graph end. Time elapsed: 15.21 s. mem usage=16.77 GB. avail mem=36.90 GB.
[2025-08-27 21:06:24] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=36.90 GB


Capturing batches (bs=1 avail_mem=36.68 GB): 100%|██████████| 4/4 [00:00<00:00, 30.39it/s]
[2025-08-27 21:06:25] Capture draft extend cuda graph end. Time elapsed: 0.81 s. mem usage=0.22 GB. avail mem=36.68 GB.
[2025-08-27 21:06:25] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=4096, available_gpu_mem=36.68 GB


[2025-08-27 21:06:25] INFO:     Started server process [1344367]
[2025-08-27 21:06:25] INFO:     Waiting for application startup.
[2025-08-27 21:06:25] INFO:     Application startup complete.
[2025-08-27 21:06:25] INFO:     Uvicorn running on http://127.0.0.1:39608 (Press CTRL+C to quit)
[2025-08-27 21:06:25] INFO:     127.0.0.1:51754 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-27 21:06:26] INFO:     127.0.0.1:51144 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-27 21:06:26] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-27 21:06:29] INFO:     127.0.0.1:51148 - "POST /generate HTTP/1.1" 200 OK
[2025-08-27 21:06:29] The server is fired up and ready to roll!


In [6]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-27 21:06:30] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-27 21:06:31] INFO:     127.0.0.1:51158 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [7]:
terminate_process(server_process)

### EAGLE-2 Decoding via Frequency-Ranked Speculative Sampling

By employing a truncated high-frequency token vocabulary in the draft model, Eagle speculative decoding reduces `lm_head` computational overhead while accelerating the pipeline without quality degradation. For more details, checkout [the paper](https://arxiv.org/pdf/arXiv:2502.14856).

In our implementation, set `--speculative-token-map` to enable the optimization. You can get the high-frequency token in FR-Spec from [this model](https://huggingface.co/thunlp/LLaMA3-Instruct-8B-FR-Spec). Or you can obtain high-frequency token by directly downloading these token from [this repo](https://github.com/thunlp/FR-Spec/tree/main?tab=readme-ov-file#prepare-fr-spec-vocabulary-subset).

Thanks for the contribution from [Weilin Zhao](https://github.com/Achazwl) and [Zhousx](https://github.com/Zhou-sx). 

In [8]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Meta-Llama-3-8B-Instruct --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-LLaMA3-Instruct-8B --speculative-num-steps 5 \
    --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --speculative-token-map thunlp/LLaMA3-Instruct-8B-FR-Spec/freq_32768.pt \
    --mem-fraction 0.7 --cuda-graph-max-bs 2 --dtype float16 
"""
)

wait_for_server(f"http://localhost:{port}")

W0827 21:06:38.723000 1354073 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0827 21:06:38.723000 1354073 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-27 21:06:40] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=31282, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='float16', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.7, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=1069003989

[2025-08-27 21:06:40] Using default HuggingFace chat template with detected content format: string


W0827 21:06:48.378000 1354583 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0827 21:06:48.378000 1354583 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0827 21:06:48.414000 1354584 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0827 21:06:48.414000 1354584 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-27 21:06:49] Casting torch.bfloat16 to torch.float16.


...

Capturing batches (bs=1 avail_mem=20.10 GB): 100%|██████████| 4/4 [00:06<00:00,  1.69s/it]
[2025-08-27 21:07:17] Capture draft cuda graph end. Time elapsed: 7.22 s. mem usage=12.67 GB. avail mem=8.95 GB.
[2025-08-27 21:07:17] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=8.95 GB


Capturing batches (bs=1 avail_mem=5.52 GB): 100%|██████████| 4/4 [00:00<00:00, 69.92it/s]
[2025-08-27 21:07:17] Capture draft extend cuda graph end. Time elapsed: 0.62 s. mem usage=3.43 GB. avail mem=5.52 GB.
[2025-08-27 21:07:17] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=8192, available_gpu_mem=5.52 GB


[2025-08-27 21:07:18] INFO:     Started server process [1354073]
[2025-08-27 21:07:18] INFO:     Waiting for application startup.
[2025-08-27 21:07:18] INFO:     Application startup complete.
[2025-08-27 21:07:18] INFO:     Uvicorn running on http://127.0.0.1:31282 (Press CTRL+C to quit)


[2025-08-27 21:07:18] INFO:     127.0.0.1:49366 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-27 21:07:19] INFO:     127.0.0.1:49382 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-27 21:07:19] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-27 21:07:19] INFO:     127.0.0.1:49388 - "POST /generate HTTP/1.1" 200 OK
[2025-08-27 21:07:19] The server is fired up and ready to roll!


In [9]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-27 21:07:23] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-27 21:07:24] INFO:     127.0.0.1:49390 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [10]:
terminate_process(server_process)

### EAGLE-3 Decoding

You can enable EAGLE-3 decoding by setting `--speculative-algorithm EAGLE3` and choosing an appropriate model.

In [11]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-3.1-8B-Instruct  --speculative-algorithm EAGLE3 \
    --speculative-draft-model-path jamesliu1/sglang-EAGLE3-Llama-3.1-Instruct-8B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 32 --mem-fraction 0.6 \
        --cuda-graph-max-bs 2 --dtype float16
"""
)

wait_for_server(f"http://localhost:{port}")

W0827 21:07:32.137000 1357206 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0827 21:07:32.137000 1357206 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-27 21:07:32] server_args=ServerArgs(model_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=39075, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='float16', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.6, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=642155958, const

[2025-08-27 21:07:33] Using default HuggingFace chat template with detected content format: string


W0827 21:07:41.347000 1357456 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0827 21:07:41.347000 1357456 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0827 21:07:41.371000 1357457 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0827 21:07:41.371000 1357457 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-27 21:07:42] Casting torch.bfloat16 to torch.float16.


...

Capturing batches (bs=1 avail_mem=53.85 GB): 100%|██████████| 4/4 [00:05<00:00,  1.28s/it]
[2025-08-27 21:08:08] Capture draft cuda graph end. Time elapsed: 5.61 s. mem usage=-0.20 GB. avail mem=56.05 GB.
[2025-08-27 21:08:08] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=56.05 GB


Capturing batches (bs=1 avail_mem=55.94 GB): 100%|██████████| 4/4 [00:00<00:00, 89.82it/s]
[2025-08-27 21:08:09] Capture draft extend cuda graph end. Time elapsed: 0.81 s. mem usage=0.11 GB. avail mem=55.94 GB.
[2025-08-27 21:08:09] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=131072, available_gpu_mem=55.85 GB


[2025-08-27 21:08:09] INFO:     Started server process [1357206]
[2025-08-27 21:08:09] INFO:     Waiting for application startup.
[2025-08-27 21:08:09] INFO:     Application startup complete.
[2025-08-27 21:08:09] INFO:     Uvicorn running on http://127.0.0.1:39075 (Press CTRL+C to quit)


[2025-08-27 21:08:10] INFO:     127.0.0.1:42116 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-27 21:08:10] INFO:     127.0.0.1:42118 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-27 21:08:10] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-27 21:08:10] INFO:     127.0.0.1:42134 - "POST /generate HTTP/1.1" 200 OK
[2025-08-27 21:08:10] The server is fired up and ready to roll!


In [12]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-27 21:08:15] Prefill batch. #new-seq: 1, #new-token: 42, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-27 21:08:15] INFO:     127.0.0.1:42144 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [13]:
terminate_process(server_process)

## Multi Token Prediction

We support [MTP(Multi-Token Prediction)](https://arxiv.org/pdf/2404.19737) in SGLang by using speculative decoding. We use Xiaomi/MiMo-7B-RL model as example here (deepseek mtp usage refer to [deepseek doc](../references/deepseek.md#multi-token-prediction))

In [14]:
server_process, port = launch_server_cmd(
    """
    python3 -m sglang.launch_server --model-path XiaomiMiMo/MiMo-7B-RL --host 0.0.0.0 --trust-remote-code \
    --speculative-algorithm EAGLE --speculative-num-steps 1 --speculative-eagle-topk 1 --speculative-num-draft-tokens 2 \
    --mem-fraction 0.5
"""
)

wait_for_server(f"http://localhost:{port}")

W0827 21:08:24.111000 1360560 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0827 21:08:24.111000 1360560 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-27 21:08:26] server_args=ServerArgs(model_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=True, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='0.0.0.0', port=30149, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.5, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=404800076, constrained_json_whitespace_patte

[2025-08-27 21:08:27] Using default HuggingFace chat template with detected content format: string


W0827 21:08:36.636000 1361412 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0827 21:08:36.636000 1361412 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0827 21:08:36.663000 1361411 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0827 21:08:36.663000 1361411 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-27 21:08:38] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-08-27 21:08:38] Init torch distributed begin.


...

Capturing batches (bs=1 avail_mem=57.21 GB): 100%|██████████| 4/4 [00:02<00:00,  1.94it/s]
[2025-08-27 21:08:49] Capture draft cuda graph end. Time elapsed: 2.51 s. mem usage=-4.98 GB. avail mem=57.21 GB.
[2025-08-27 21:08:49] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=57.21 GB


Capturing batches (bs=1 avail_mem=59.35 GB): 100%|██████████| 4/4 [00:00<00:00, 52.73it/s]
[2025-08-27 21:08:49] Capture draft extend cuda graph end. Time elapsed: 0.62 s. mem usage=-2.15 GB. avail mem=59.35 GB.
[2025-08-27 21:08:49] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=32768, available_gpu_mem=59.35 GB


[2025-08-27 21:08:50] INFO:     Started server process [1360560]
[2025-08-27 21:08:50] INFO:     Waiting for application startup.
[2025-08-27 21:08:50] INFO:     Application startup complete.
[2025-08-27 21:08:50] INFO:     Uvicorn running on http://0.0.0.0:30149 (Press CTRL+C to quit)


[2025-08-27 21:08:50] INFO:     127.0.0.1:56854 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-27 21:08:51] INFO:     127.0.0.1:56856 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-27 21:08:51] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-27 21:08:51] INFO:     127.0.0.1:56866 - "POST /generate HTTP/1.1" 200 OK
[2025-08-27 21:08:51] The server is fired up and ready to roll!


In [15]:
import requests

url = f"http://localhost:{port}/v1/chat/completions"

data = {
    "model": "XiaomiMiMo/MiMo-7B-RL",
    "messages": [{"role": "user", "content": "What is the capital of France?"}],
}

response = requests.post(url, json=data)
print_highlight(response.json())

[2025-08-27 21:08:55] Prefill batch. #new-seq: 1, #new-token: 26, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-27 21:08:56] Decode batch. #running-req: 1, #token: 87, token usage: 0.00, accept len: 1.67, cuda graph: True, gen throughput (token/s): 10.34, #queue-req: 0, 


[2025-08-27 21:08:56] Decode batch. #running-req: 1, #token: 151, token usage: 0.01, accept len: 1.60, cuda graph: True, gen throughput (token/s): 163.93, #queue-req: 0, 


[2025-08-27 21:08:57] Decode batch. #running-req: 1, #token: 221, token usage: 0.01, accept len: 1.75, cuda graph: True, gen throughput (token/s): 179.06, #queue-req: 0, 


[2025-08-27 21:08:57] Decode batch. #running-req: 1, #token: 288, token usage: 0.01, accept len: 1.68, cuda graph: True, gen throughput (token/s): 171.19, #queue-req: 0, 


[2025-08-27 21:08:57] Decode batch. #running-req: 1, #token: 355, token usage: 0.02, accept len: 1.68, cuda graph: True, gen throughput (token/s): 170.89, #queue-req: 0, 


[2025-08-27 21:08:58] Decode batch. #running-req: 1, #token: 425, token usage: 0.02, accept len: 1.75, cuda graph: True, gen throughput (token/s): 178.34, #queue-req: 0, 


[2025-08-27 21:08:58] INFO:     127.0.0.1:38890 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [16]:
terminate_process(server_process)

## References

EAGLE process is as follows:

- Within EAGLE the draft model predicts the next feature vector, i.e. the last hidden state of the original LLM, using the feature sequence $(f_1, ..., f_k)$ and the token sequence $(t_2, ..., t_{k+1})$. 
- The next token is then sampled from $p_{k+2}=\text{LMHead}(f_{k+1})$. Afterwards, the two sequences are extended in a tree style—branching out multiple potential continuations, with the branching factor per step controlled by the `speculative_eagle_topk` parameter—to ensure a more coherent connection of context, and are given as input again.
- EAGLE-2 additionally uses the draft model to evaluate how probable certain branches in the draft tree are, dynamically stopping the expansion of unlikely branches. After the expansion phase, reranking is employed to select only the top `speculative_num_draft_tokens` final nodes as draft tokens.
- EAGLE-3 removes the feature prediction objective, incorporates low and mid-layer features, and is trained in an on-policy manner.

This enhances drafting accuracy by operating on the features instead of tokens for more regular inputs and passing the tokens from the next timestep additionally to minimize randomness effects from sampling. Furthermore the dynamic adjustment of the draft tree and selection of reranked final nodes increases acceptance rate of draft tokens further. For more details see [EAGLE-2](https://arxiv.org/abs/2406.16858) and [EAGLE-3](https://arxiv.org/abs/2503.01840) paper.


For guidance how to train your own EAGLE model please see the [EAGLE repo](https://github.com/SafeAILab/EAGLE/tree/main?tab=readme-ov-file#train).